# GCN architecture and hyperparameter tuning
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt
from time import time

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


## 2. Filter

In [3]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Grid search over architecture, learning rate, heads

In [38]:
saveDir = './results/modelTuning/architectureTuning02/'

# build training plan
architectureList = ['L16/C32/C64/C128/C256/C256/C128/L64/L2',
                    'L16/C32/C64/C128/C256/C512/C256/C128/L64/L2',
                    'L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2']
learningRatelist = [1e-4,1e-3,1e-2]
headList = [4, 8, 12]
seedList = [1,2,3]
trainConfigs = []
for ar in architectureList:
    for lr in learningRatelist:
        for he in headList:
            for s in seedList:
                trainConfigs.append({'Architecture': ar, 'Learning Rate': lr, '# of Heads': he, 'Trial': s})


# train models
for i,c in enumerate(trainConfigs):
    # partition data
    trainData, valData, testData = partitionGraphList(allGraphs, seed=c['Trial'])
    
    print(c)
    gcn = FeaStNet(architecture=c['Architecture'], heads=c['# of Heads'], useConstraints=False)
    c['Parameters'] = gcn.getNumParams()
    
    t = time()
    history = gcn.trainModel(trainData, valData, 
                             epochs=100, 
                             lr=c['Learning Rate'],
                             batchSize=256, 
                             flatten=True, 
                             logTrans=False, 
                             ssTrans=True, 
                             saveDir=saveDir+f'/{i:03}/')
    c['Train Time'] = time()-t
    plotHistory(history)
    
    # test
    trainRes = gcn.testModel(trainData)
    c['Train MSE'] = trainRes['mse']
    
    testRes = gcn.testModel(testData)
    c['Test MSE'] = testRes['mse']

    # log
    df = pd.DataFrame(trainConfigs)
    df.to_csv(saveDir+'testResults.csv', index=False)

{'Architecture': 'L16/C32/C64/C128/C256/C256/C128/L64/L2', 'Learning Rate': 0.0001, '# of Heads': 4, 'Trial': 1}
epoch: 0   trainLoss: 1.0154e+00   valLoss:9.2225e-01  time: 1.11e+00
epoch: 1   trainLoss: 8.7191e-01   valLoss:9.1944e-01  time: 9.48e-01
epoch: 2   trainLoss: 8.3652e-01   valLoss:9.1786e-01  time: 9.16e-01
epoch: 3   trainLoss: 8.0255e-01   valLoss:9.1919e-01  time: 9.20e-01
epoch: 4   trainLoss: 7.7544e-01   valLoss:8.8549e-01  time: 9.29e-01
epoch: 5   trainLoss: 7.6053e-01   valLoss:8.2063e-01  time: 9.17e-01
epoch: 6   trainLoss: 7.1038e-01   valLoss:7.5505e-01  time: 9.19e-01
epoch: 7   trainLoss: 6.7002e-01   valLoss:6.9353e-01  time: 9.17e-01
epoch: 8   trainLoss: 6.5622e-01   valLoss:6.5124e-01  time: 9.16e-01
epoch: 9   trainLoss: 6.3628e-01   valLoss:6.2276e-01  time: 9.26e-01
epoch: 10   trainLoss: 5.9700e-01   valLoss:6.0029e-01  time: 9.29e-01
epoch: 11   trainLoss: 6.0433e-01   valLoss:5.9013e-01  time: 9.22e-01
epoch: 12   trainLoss: 5.3116e-01   valLoss:5

epoch: 13   trainLoss: 5.0701e-01   valLoss:4.9287e-01  time: 9.16e-01
epoch: 14   trainLoss: 5.3027e-01   valLoss:4.8047e-01  time: 9.24e-01
epoch: 15   trainLoss: 5.0274e-01   valLoss:4.6555e-01  time: 9.20e-01
epoch: 16   trainLoss: 4.8123e-01   valLoss:4.5257e-01  time: 9.26e-01
epoch: 17   trainLoss: 4.6662e-01   valLoss:4.4738e-01  time: 9.22e-01
epoch: 18   trainLoss: 4.4607e-01   valLoss:4.4128e-01  time: 9.35e-01
epoch: 19   trainLoss: 4.2427e-01   valLoss:4.2423e-01  time: 9.41e-01
epoch: 20   trainLoss: 4.2263e-01   valLoss:4.1001e-01  time: 9.26e-01
epoch: 21   trainLoss: 4.1657e-01   valLoss:3.9446e-01  time: 9.34e-01
epoch: 22   trainLoss: 4.1287e-01   valLoss:3.8720e-01  time: 9.35e-01
epoch: 23   trainLoss: 4.0507e-01   valLoss:3.8414e-01  time: 9.19e-01
epoch: 24   trainLoss: 3.6481e-01   valLoss:3.6817e-01  time: 9.36e-01
epoch: 25   trainLoss: 3.5452e-01   valLoss:3.5765e-01  time: 9.28e-01
epoch: 26   trainLoss: 3.5458e-01   valLoss:3.5780e-01  time: 9.33e-01
epoch:

epoch: 27   trainLoss: 3.3185e-01   valLoss:3.2402e-01  time: 9.28e-01
epoch: 28   trainLoss: 3.2757e-01   valLoss:3.1150e-01  time: 9.25e-01
epoch: 29   trainLoss: 2.9525e-01   valLoss:2.9490e-01  time: 9.21e-01
epoch: 30   trainLoss: 3.2458e-01   valLoss:2.8892e-01  time: 9.23e-01
epoch: 31   trainLoss: 3.1854e-01   valLoss:2.8929e-01  time: 9.20e-01
epoch: 32   trainLoss: 2.8351e-01   valLoss:2.7258e-01  time: 9.19e-01
epoch: 33   trainLoss: 2.7451e-01   valLoss:2.6974e-01  time: 9.26e-01
epoch: 34   trainLoss: 2.4390e-01   valLoss:2.6715e-01  time: 9.37e-01
epoch: 35   trainLoss: 2.5390e-01   valLoss:2.6824e-01  time: 9.36e-01
epoch: 36   trainLoss: 2.4079e-01   valLoss:2.4475e-01  time: 9.28e-01
epoch: 37   trainLoss: 2.5813e-01   valLoss:2.4483e-01  time: 9.20e-01
epoch: 38   trainLoss: 2.2818e-01   valLoss:2.4030e-01  time: 9.38e-01
epoch: 39   trainLoss: 2.2057e-01   valLoss:2.3448e-01  time: 9.31e-01
epoch: 40   trainLoss: 2.2812e-01   valLoss:2.2428e-01  time: 9.28e-01
epoch:

epoch: 41   trainLoss: 2.0401e-01   valLoss:2.8549e-01  time: 1.03e+00
epoch: 42   trainLoss: 1.9689e-01   valLoss:2.6647e-01  time: 1.03e+00
epoch: 43   trainLoss: 1.9697e-01   valLoss:2.5673e-01  time: 1.02e+00
epoch: 44   trainLoss: 1.9275e-01   valLoss:2.4684e-01  time: 1.06e+00
epoch: 45   trainLoss: 1.8394e-01   valLoss:2.4719e-01  time: 1.03e+00
epoch: 46   trainLoss: 1.7250e-01   valLoss:2.4132e-01  time: 1.04e+00
epoch: 47   trainLoss: 1.7931e-01   valLoss:2.3088e-01  time: 1.04e+00
epoch: 48   trainLoss: 1.6774e-01   valLoss:2.2946e-01  time: 1.04e+00
epoch: 49   trainLoss: 1.6487e-01   valLoss:2.2779e-01  time: 1.04e+00
epoch: 50   trainLoss: 1.5387e-01   valLoss:2.2342e-01  time: 1.02e+00
epoch: 51   trainLoss: 1.6630e-01   valLoss:2.3495e-01  time: 1.03e+00
epoch: 52   trainLoss: 1.4550e-01   valLoss:2.2595e-01  time: 1.04e+00
epoch: 53   trainLoss: 1.3769e-01   valLoss:2.1142e-01  time: 1.03e+00
epoch: 54   trainLoss: 1.4507e-01   valLoss:2.2713e-01  time: 1.05e+00
epoch:

epoch: 55   trainLoss: 1.2531e-01   valLoss:1.9316e-01  time: 1.03e+00
epoch: 56   trainLoss: 1.2673e-01   valLoss:1.9355e-01  time: 1.04e+00
epoch: 57   trainLoss: 1.3074e-01   valLoss:1.8363e-01  time: 1.03e+00
epoch: 58   trainLoss: 1.1792e-01   valLoss:1.7886e-01  time: 1.02e+00
epoch: 59   trainLoss: 1.1381e-01   valLoss:1.9052e-01  time: 1.03e+00
epoch: 60   trainLoss: 1.0801e-01   valLoss:1.8230e-01  time: 1.02e+00
epoch: 61   trainLoss: 1.0921e-01   valLoss:1.7403e-01  time: 1.02e+00
epoch: 62   trainLoss: 1.2344e-01   valLoss:1.7897e-01  time: 1.03e+00
epoch: 63   trainLoss: 1.0265e-01   valLoss:1.8234e-01  time: 1.02e+00
epoch: 64   trainLoss: 1.0247e-01   valLoss:1.6918e-01  time: 1.02e+00
epoch: 65   trainLoss: 1.1272e-01   valLoss:1.6756e-01  time: 1.05e+00
epoch: 66   trainLoss: 1.0736e-01   valLoss:1.7168e-01  time: 1.04e+00
epoch: 67   trainLoss: 1.0125e-01   valLoss:1.8549e-01  time: 1.02e+00
epoch: 68   trainLoss: 1.0219e-01   valLoss:1.7413e-01  time: 1.04e+00
epoch:

...

epoch: 20   trainLoss: 5.9391e-01   valLoss:5.2485e-01  time: 1.85e+00
epoch: 21   trainLoss: 4.8568e-01   valLoss:6.0074e-01  time: 1.78e+00
epoch: 22   trainLoss: 3.8754e-01   valLoss:6.4644e-01  time: 1.79e+00
epoch: 23   trainLoss: 3.0349e-01   valLoss:7.9758e-01  time: 1.81e+00
epoch: 24   trainLoss: 4.0397e-01   valLoss:8.7364e-01  time: 1.83e+00
epoch: 25   trainLoss: 3.6598e-01   valLoss:5.7264e-01  time: 1.80e+00
epoch: 26   trainLoss: 3.5328e-01   valLoss:5.1938e-01  time: 1.83e+00
epoch: 27   trainLoss: 2.9871e-01   valLoss:9.0669e-01  time: 1.80e+00
epoch: 28   trainLoss: 3.5806e-01   valLoss:4.8273e-01  time: 1.83e+00
epoch: 29   trainLoss: 3.3156e-01   valLoss:4.9548e-01  time: 1.83e+00
epoch: 30   trainLoss: 3.0881e-01   valLoss:5.6650e-01  time: 1.98e+00
epoch: 31   trainLoss: 3.3225e-01   valLoss:6.1349e-01  time: 1.94e+00
epoch: 32   trainLoss: 2.8014e-01   valLoss:1.4364e+00  time: 1.96e+00
epoch: 33   trainLoss: 3.2667e-01   valLoss:9.3299e-01  time: 1.84e+00
epoch:

epoch: 34   trainLoss: 2.9387e-01   valLoss:6.6115e-01  time: 1.82e+00
epoch: 35   trainLoss: 3.4184e-01   valLoss:5.1207e-01  time: 1.84e+00
epoch: 36   trainLoss: 3.1845e-01   valLoss:3.4942e-01  time: 1.82e+00
epoch: 37   trainLoss: 2.6874e-01   valLoss:5.5716e-01  time: 1.82e+00
epoch: 38   trainLoss: 2.5251e-01   valLoss:6.2654e-01  time: 1.82e+00
epoch: 39   trainLoss: 2.6348e-01   valLoss:8.0213e-01  time: 1.82e+00
epoch: 40   trainLoss: 2.6711e-01   valLoss:5.1832e-01  time: 1.82e+00
epoch: 41   trainLoss: 2.2905e-01   valLoss:1.1757e+00  time: 1.82e+00
epoch: 42   trainLoss: 2.0887e-01   valLoss:1.6063e+00  time: 1.82e+00
epoch: 43   trainLoss: 2.5699e-01   valLoss:6.4930e-01  time: 1.81e+00
epoch: 44   trainLoss: 2.2679e-01   valLoss:3.6342e-01  time: 1.79e+00
epoch: 45   trainLoss: 2.4383e-01   valLoss:2.8734e-01  time: 1.81e+00
epoch: 46   trainLoss: 2.2998e-01   valLoss:3.6846e-01  time: 1.82e+00
epoch: 47   trainLoss: 1.9558e-01   valLoss:3.2225e-01  time: 1.80e+00
epoch:

epoch: 48   trainLoss: 2.7528e-01   valLoss:2.9859e-01  time: 2.22e+00
epoch: 49   trainLoss: 2.4690e-01   valLoss:4.7920e-01  time: 2.20e+00
epoch: 50   trainLoss: 2.6742e-01   valLoss:4.6124e-01  time: 2.19e+00
epoch: 51   trainLoss: 2.2918e-01   valLoss:3.6456e-01  time: 2.19e+00
epoch: 52   trainLoss: 2.0773e-01   valLoss:3.3083e-01  time: 2.18e+00
epoch: 53   trainLoss: 1.8534e-01   valLoss:7.1191e-01  time: 2.18e+00
epoch: 54   trainLoss: 2.3440e-01   valLoss:4.2580e-01  time: 2.19e+00
epoch: 55   trainLoss: 2.5029e-01   valLoss:4.6494e-01  time: 2.21e+00
epoch: 56   trainLoss: 1.9777e-01   valLoss:4.3044e-01  time: 2.20e+00
epoch: 57   trainLoss: 1.6819e-01   valLoss:6.3567e-01  time: 2.19e+00
epoch: 58   trainLoss: 1.7287e-01   valLoss:5.7071e-01  time: 2.20e+00
epoch: 59   trainLoss: 2.0121e-01   valLoss:8.3348e-01  time: 2.19e+00
epoch: 60   trainLoss: 2.2172e-01   valLoss:5.1508e-01  time: 2.20e+00
epoch: 61   trainLoss: 2.2853e-01   valLoss:2.9284e-01  time: 2.19e+00
epoch:

epoch: 62   trainLoss: 1.8896e-01   valLoss:5.1767e-01  time: 2.18e+00
epoch: 63   trainLoss: 1.8353e-01   valLoss:3.1934e-01  time: 2.19e+00
epoch: 64   trainLoss: 2.3171e-01   valLoss:3.8700e-01  time: 2.18e+00
epoch: 65   trainLoss: 3.6663e-01   valLoss:1.0702e+00  time: 2.20e+00
epoch: 66   trainLoss: 2.9975e-01   valLoss:4.2061e-01  time: 2.19e+00
epoch: 67   trainLoss: 2.6473e-01   valLoss:1.0137e+00  time: 2.17e+00
epoch: 68   trainLoss: 2.6420e-01   valLoss:3.2427e-01  time: 2.19e+00
epoch: 69   trainLoss: 1.9651e-01   valLoss:9.3034e-01  time: 2.18e+00
epoch: 70   trainLoss: 1.9625e-01   valLoss:2.4343e-01  time: 2.18e+00
epoch: 71   trainLoss: 1.7105e-01   valLoss:3.6976e-01  time: 2.18e+00
epoch: 72   trainLoss: 1.8873e-01   valLoss:4.3800e-01  time: 2.18e+00
epoch: 73   trainLoss: 1.6702e-01   valLoss:5.1504e-01  time: 2.18e+00
epoch: 74   trainLoss: 1.8017e-01   valLoss:3.2408e-01  time: 2.18e+00
epoch: 75   trainLoss: 1.7905e-01   valLoss:2.9483e-01  time: 2.20e+00
epoch:

epoch: 76   trainLoss: 1.2837e-01   valLoss:1.0498e+00  time: 2.20e+00
epoch: 77   trainLoss: 1.4836e-01   valLoss:4.7093e-01  time: 2.34e+00
epoch: 78   trainLoss: 1.2786e-01   valLoss:4.1503e-01  time: 2.33e+00
epoch: 79   trainLoss: 1.8444e-01   valLoss:6.6615e-01  time: 2.24e+00
epoch: 80   trainLoss: 2.7550e-01   valLoss:3.2099e-01  time: 2.18e+00
epoch: 81   trainLoss: 2.1187e-01   valLoss:3.1734e-01  time: 2.18e+00
epoch: 82   trainLoss: 1.7767e-01   valLoss:2.3749e+00  time: 2.35e+00
epoch: 83   trainLoss: 3.5292e-01   valLoss:2.0376e+00  time: 2.36e+00
epoch: 84   trainLoss: 2.0615e-01   valLoss:5.9324e-01  time: 2.34e+00
epoch: 85   trainLoss: 1.9670e-01   valLoss:2.0010e-01  time: 2.34e+00
epoch: 86   trainLoss: 1.5320e-01   valLoss:4.0634e-01  time: 2.33e+00
epoch: 87   trainLoss: 1.7508e-01   valLoss:2.8861e-01  time: 2.20e+00
epoch: 88   trainLoss: 1.4600e-01   valLoss:2.0073e-01  time: 2.34e+00
epoch: 89   trainLoss: 1.7229e-01   valLoss:3.0226e-01  time: 2.35e+00
epoch:

In [46]:
# 27 configurations averaged over three trials
df.groupby(['Architecture', 'Learning Rate', '# of Heads']).mean()[['Parameters', 'Train Time','Train MSE', 'Test MSE']]

Parameters  \
Architecture                                     Learning Rate # of Heads               
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0001        4               581358   
                                                               8              1151686   
                                                               12             1722014   
                                                 0.0010        4               581358   
                                                               8              1151686   
                                                               12             1722014   
                                                 0.0100        4               581358   
                                                               8              1151686   
                                                               12             1722014   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0001        4              1371378   
                                                               8              2730190   
                                                               12             4089002   
                                                 0.0010        4              1371378   
                                                               8              2730190   
                                                               12             4089002   
                                                 0.0100        4              1371378   
                                                               8              2730190   
                                                               12             4089002   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 0.0001        4              2423542   
                                                               8              4832982   
                                                               12             7242422   
                                                 0.0010        4              2423542   
                                                               8              4832982   
                                                               12             7242422   
                                                 0.0100        4              2423542   
                                                               8              4832982   
                                                               12             7242422   

                                                                           Train Time  \
Architecture                                     Learning Rate # of Heads               
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0001        4            94.495845   
                                                               8           104.668510   
                                                               12          115.668040   
                                                 0.0010        4            96.475638   
                                                               8           104.174627   
                                                               12          114.658639   
                                                 0.0100        4            92.937883   
                                                               8           103.734295   
                                                               12          114.905799   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0001        4           118.563231   
                                                               8           141.981915   
                                                               12          165.990062   
                                                 0.0010        4           117.414840   
                                                               8           146.611935   
                                                               12    

In [48]:
# sorted
df.groupby(['Architecture', 'Learning Rate', '# of Heads']).mean().sort_values('Test MSE')[['Parameters', 'Train Time','Train MSE', 'Test MSE']]

Parameters  \
Architecture                                     Learning Rate # of Heads               
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0010        8              2730190   
                                                               4              1371378   
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0010        8              1151686   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 0.0010        8              4832982   
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0010        12             1722014   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0010        12             4089002   
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0010        4               581358   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 0.0010        12             7242422   
                                                               4              2423542   
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0100        4               581358   
                                                 0.0001        12             1722014   
                                                 0.0100        12             1722014   
                                                               8              1151686   
                                                 0.0001        8              1151686   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0001        12             4089002   
                                                               8              2730190   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 0.0001        4              2423542   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0001        4              1371378   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 0.0100        4              2423542   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0100        4              1371378   
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0001        4               581358   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0100        8              2730190   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 0.0100        8              4832982   
                                                 0.0001        8              4832982   
                                                               12             7242422   
                                                 0.0100        12             7242422   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0100        12             4089002   

                                                                           Train Time  \
Architecture                                     Learning Rate # of Heads               
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0010        8           146.611935   
                                                               4           117.414840   
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0010        8           104.174627   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 0.0010        8           188.563491   
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0010        12          114.658639   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0010        12          161.624589   
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0010        4            96.475638   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 0.0010        12          223.382841   
                                                               4           150.439876   
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0100        4            92.937883   
                                                 0.0001        12          115.668040   
                                                 0.0100        12          114.905799   
                                                               8           103.734295   
                                                 0.0001        8           104.668510   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0001        12    

In [40]:
df.groupby(['Architecture']).mean()

,Learning Rate,# of Heads,Trial,Parameters,Train Time,Train MSE,Test MSE
Architecture,,,,,,,
L16/C32/C64/C128/C256/C256/C128/L64/L2,0.0037,8,2,1151686,104.635475,0.000015,0.000026
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2,0.0037,8,2,2730190,141.974837,0.000016,0.000030
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2,0.0037,8,2,4832982,185.815946,0.000017,0.000032


In [49]:
df.groupby(['Learning Rate']).mean()

,# of Heads,Trial,Parameters,Train Time,Train MSE,Test MSE
Learning Rate,,,,,,
0.0001,8.0,2.0,2.904953e+06,144.202396,0.000011,0.000033
0.0010,8.0,2.0,2.904953e+06,144.816275,0.000009,0.000020
0.0100,8.0,2.0,2.904953e+06,143.407588,0.000028,0.000035


## 4. Study data transformations

In [12]:
saveDir = './results/modelTuning/transformTuning01/'

# build training plan
options = [True, False]
seedList = [1,2,3]
trainConfigs = []
for st in options:
    for lt in options:
        for s in seedList:
            trainConfigs.append({'Standard Scaler': st, 'Log Transform': lt, 'Trial': s})

# train models
for i,c in enumerate(trainConfigs):
    print(c)
    trainData, valData, testData = partitionGraphList(allGraphs, seed=c['Trial'])
    gcn = FeaStNet(architecture='L16/C32/C64/C128/C256/C512/C256/C128/L64/L2', heads=8, useConstraints=False)
    
    t = time()
    history = gcn.trainModel(trainData, valData, 
                             epochs=100,
                             lr=1e-3,
                             batchSize=256, 
                             flatten=True, 
                             logTrans=c['Log Transform'], 
                             ssTrans=c['Standard Scaler'], 
                             saveDir=saveDir+f'/{i:03}/')
    
    # test
    trainRes = gcn.testModel(trainData)
    c['Train MSE'] = trainRes['mse']
    
    testRes = gcn.testModel(testData)
    c['Test MSE'] = testRes['mse']

    # log
    df = pd.DataFrame(trainConfigs)
    df.to_csv(saveDir+'testResults.csv', index=False)

{'Standard Scaler': True, 'Log Transform': True, 'Trial': 1}
epoch: 0   trainLoss: 3.3730e+00   valLoss:3.2823e+00  time: 1.51e+00
epoch: 1   trainLoss: 2.8218e+00   valLoss:3.0822e+00  time: 1.39e+00
epoch: 2   trainLoss: 2.2056e+00   valLoss:3.1703e+00  time: 1.42e+00
epoch: 3   trainLoss: 1.7758e+00   valLoss:3.5928e+00  time: 1.42e+00
epoch: 4   trainLoss: 1.6130e+00   valLoss:4.2503e+00  time: 1.42e+00
epoch: 5   trainLoss: 1.3020e+00   valLoss:5.3417e+00  time: 1.42e+00
epoch: 6   trainLoss: 1.0758e+00   valLoss:4.0878e+00  time: 1.42e+00
epoch: 7   trainLoss: 8.7072e-01   valLoss:1.8933e+00  time: 1.41e+00
epoch: 8   trainLoss: 7.2386e-01   valLoss:1.3619e+00  time: 1.40e+00
epoch: 9   trainLoss: 6.6876e-01   valLoss:1.1203e+00  time: 1.40e+00
epoch: 10   trainLoss: 5.4587e-01   valLoss:1.4838e+00  time: 1.40e+00
epoch: 11   trainLoss: 5.4358e-01   valLoss:1.4048e+00  time: 1.40e+00
epoch: 12   trainLoss: 6.1722e-01   valLoss:1.3492e+00  time: 1.40e+00
epoch: 13   trainLoss: 5.4

epoch: 14   trainLoss: 5.5659e-01   valLoss:1.6764e+00  time: 1.36e+00
epoch: 15   trainLoss: 4.5251e-01   valLoss:1.1733e+00  time: 1.37e+00
epoch: 16   trainLoss: 4.9012e-01   valLoss:1.2659e+00  time: 1.37e+00
epoch: 17   trainLoss: 3.7525e-01   valLoss:1.1178e+00  time: 1.36e+00
epoch: 18   trainLoss: 3.6125e-01   valLoss:8.2096e-01  time: 1.37e+00
epoch: 19   trainLoss: 3.5300e-01   valLoss:8.3378e-01  time: 1.38e+00
epoch: 20   trainLoss: 3.0137e-01   valLoss:7.4509e-01  time: 1.39e+00
epoch: 21   trainLoss: 3.6224e-01   valLoss:1.0737e+00  time: 1.36e+00
epoch: 22   trainLoss: 4.2916e-01   valLoss:6.1926e-01  time: 1.36e+00
epoch: 23   trainLoss: 2.9637e-01   valLoss:1.3997e+00  time: 1.38e+00
epoch: 24   trainLoss: 2.5580e-01   valLoss:1.8752e+00  time: 1.36e+00
epoch: 25   trainLoss: 2.5444e-01   valLoss:1.0204e+00  time: 1.35e+00
epoch: 26   trainLoss: 2.5702e-01   valLoss:1.4832e+00  time: 1.36e+00
epoch: 27   trainLoss: 2.8049e-01   valLoss:5.8213e-01  time: 1.37e+00
epoch:

epoch: 29   trainLoss: 2.2512e-01   valLoss:1.1367e+00  time: 1.39e+00
epoch: 30   trainLoss: 2.0057e-01   valLoss:8.6688e-01  time: 1.39e+00
epoch: 31   trainLoss: 1.8110e-01   valLoss:9.4434e-01  time: 1.36e+00
epoch: 32   trainLoss: 1.7755e-01   valLoss:7.6961e-01  time: 1.37e+00
epoch: 33   trainLoss: 2.0327e-01   valLoss:8.9580e-01  time: 1.37e+00
epoch: 34   trainLoss: 2.0243e-01   valLoss:1.0368e+00  time: 1.39e+00
epoch: 35   trainLoss: 1.6783e-01   valLoss:8.1700e-01  time: 1.39e+00
epoch: 36   trainLoss: 1.7186e-01   valLoss:1.1758e+00  time: 1.37e+00
epoch: 37   trainLoss: 1.5880e-01   valLoss:1.3444e+00  time: 1.36e+00
epoch: 38   trainLoss: 2.1344e-01   valLoss:7.0263e-01  time: 1.40e+00
epoch: 39   trainLoss: 1.6603e-01   valLoss:6.7847e-01  time: 1.39e+00
epoch: 40   trainLoss: 1.6204e-01   valLoss:6.1914e-01  time: 1.39e+00
epoch: 41   trainLoss: 1.6656e-01   valLoss:9.6027e-01  time: 1.39e+00
epoch: 42   trainLoss: 1.6220e-01   valLoss:7.4473e-01  time: 1.37e+00
epoch:

epoch: 44   trainLoss: 6.5102e-02   valLoss:2.0946e-01  time: 1.38e+00
epoch: 45   trainLoss: 5.4802e-02   valLoss:1.8215e-01  time: 1.38e+00
epoch: 46   trainLoss: 6.5117e-02   valLoss:1.8247e-01  time: 1.39e+00
epoch: 47   trainLoss: 4.4463e-02   valLoss:1.5408e-01  time: 1.38e+00
epoch: 48   trainLoss: 4.5099e-02   valLoss:1.3581e-01  time: 1.40e+00
epoch: 49   trainLoss: 4.7042e-02   valLoss:1.7857e-01  time: 1.39e+00
epoch: 50   trainLoss: 3.9293e-02   valLoss:1.4398e-01  time: 1.39e+00
epoch: 51   trainLoss: 3.3178e-02   valLoss:1.4915e-01  time: 1.39e+00
epoch: 52   trainLoss: 3.5938e-02   valLoss:1.0956e-01  time: 1.40e+00
epoch: 53   trainLoss: 4.4962e-02   valLoss:1.3393e-01  time: 1.40e+00
epoch: 54   trainLoss: 3.5984e-02   valLoss:1.0602e-01  time: 1.39e+00
epoch: 55   trainLoss: 7.1875e-02   valLoss:2.7155e-01  time: 1.39e+00
epoch: 56   trainLoss: 5.3789e-02   valLoss:1.7352e-01  time: 1.39e+00
epoch: 57   trainLoss: 6.1844e-02   valLoss:2.0617e-01  time: 1.38e+00
epoch:

epoch: 59   trainLoss: 5.6927e-02   valLoss:1.2672e-01  time: 1.40e+00
epoch: 60   trainLoss: 6.5265e-02   valLoss:2.6609e-01  time: 1.42e+00
epoch: 61   trainLoss: 6.2595e-02   valLoss:1.7295e-01  time: 1.38e+00
epoch: 62   trainLoss: 4.0756e-02   valLoss:1.3160e-01  time: 1.36e+00
epoch: 63   trainLoss: 5.4328e-02   valLoss:1.1451e-01  time: 1.37e+00
epoch: 64   trainLoss: 4.8023e-02   valLoss:2.2792e-01  time: 1.37e+00
epoch: 65   trainLoss: 4.2396e-02   valLoss:1.7269e-01  time: 1.38e+00
epoch: 66   trainLoss: 5.0499e-02   valLoss:1.4735e-01  time: 1.40e+00
epoch: 67   trainLoss: 4.6145e-02   valLoss:1.5920e-01  time: 1.39e+00
epoch: 68   trainLoss: 5.0126e-02   valLoss:1.3352e-01  time: 1.36e+00
epoch: 69   trainLoss: 4.6374e-02   valLoss:1.6249e-01  time: 1.36e+00
epoch: 70   trainLoss: 3.7793e-02   valLoss:1.4954e-01  time: 1.36e+00
epoch: 71   trainLoss: 4.2795e-02   valLoss:1.1312e-01  time: 1.35e+00
epoch: 72   trainLoss: 3.7242e-02   valLoss:1.5316e-01  time: 1.39e+00
epoch:

...

epoch: 89   trainLoss: 1.9575e-03   valLoss:4.0118e-03  time: 1.37e+00
epoch: 90   trainLoss: 2.0399e-03   valLoss:3.1952e-03  time: 1.38e+00
epoch: 91   trainLoss: 1.8342e-03   valLoss:3.6334e-03  time: 1.36e+00
epoch: 92   trainLoss: 2.0697e-03   valLoss:3.8438e-03  time: 1.37e+00
epoch: 93   trainLoss: 1.9407e-03   valLoss:7.3136e-03  time: 1.34e+00
epoch: 94   trainLoss: 2.2814e-03   valLoss:4.2729e-03  time: 1.37e+00
epoch: 95   trainLoss: 2.0702e-03   valLoss:3.5332e-03  time: 1.37e+00
epoch: 96   trainLoss: 2.1712e-03   valLoss:5.9708e-03  time: 1.38e+00
epoch: 97   trainLoss: 2.6865e-03   valLoss:6.9248e-03  time: 1.36e+00
epoch: 98   trainLoss: 2.0765e-03   valLoss:3.3537e-03  time: 1.35e+00
epoch: 99   trainLoss: 1.9917e-03   valLoss:3.5988e-03  time: 1.35e+00
loading checkpoint 83
{'Standard Scaler': False, 'Log Transform': True, 'Trial': 2}
epoch: 0   trainLoss: 4.3075e-02   valLoss:1.5705e-02  time: 1.45e+00
epoch: 1   trainLoss: 1.8261e-02   valLoss:1.4012e-02  time: 1.34

epoch: 3   trainLoss: 1.1113e-02   valLoss:1.4486e-02  time: 1.35e+00
epoch: 4   trainLoss: 8.8586e-03   valLoss:1.7454e-02  time: 1.36e+00
epoch: 5   trainLoss: 8.2867e-03   valLoss:1.6958e-02  time: 1.35e+00
epoch: 6   trainLoss: 7.9375e-03   valLoss:1.5161e-02  time: 1.36e+00
epoch: 7   trainLoss: 7.4290e-03   valLoss:1.2054e-02  time: 1.39e+00
epoch: 8   trainLoss: 6.5457e-03   valLoss:8.3568e-03  time: 1.34e+00
epoch: 9   trainLoss: 6.1457e-03   valLoss:7.6493e-03  time: 1.34e+00
epoch: 10   trainLoss: 6.2314e-03   valLoss:8.3969e-03  time: 1.36e+00
epoch: 11   trainLoss: 5.7527e-03   valLoss:7.7903e-03  time: 1.35e+00
epoch: 12   trainLoss: 5.1795e-03   valLoss:7.0574e-03  time: 1.36e+00
epoch: 13   trainLoss: 4.9764e-03   valLoss:7.6690e-03  time: 1.37e+00
epoch: 14   trainLoss: 4.4657e-03   valLoss:6.8962e-03  time: 1.38e+00
epoch: 15   trainLoss: 4.3818e-03   valLoss:7.8073e-03  time: 1.37e+00
epoch: 16   trainLoss: 4.3742e-03   valLoss:7.9013e-03  time: 1.37e+00
epoch: 17   t

epoch: 18   trainLoss: 2.9578e-04   valLoss:3.1324e-04  time: 1.38e+00
epoch: 19   trainLoss: 2.6317e-04   valLoss:4.1665e-04  time: 1.37e+00
epoch: 20   trainLoss: 3.0718e-04   valLoss:3.3147e-04  time: 1.37e+00
epoch: 21   trainLoss: 2.7266e-04   valLoss:4.0883e-04  time: 1.36e+00
epoch: 22   trainLoss: 2.9220e-04   valLoss:3.2682e-04  time: 1.38e+00
epoch: 23   trainLoss: 3.2008e-04   valLoss:4.0214e-04  time: 1.36e+00
epoch: 24   trainLoss: 2.9862e-04   valLoss:3.1881e-04  time: 1.37e+00
epoch: 25   trainLoss: 3.0974e-04   valLoss:3.1123e-04  time: 1.39e+00
epoch: 26   trainLoss: 2.9072e-04   valLoss:2.5207e-04  time: 1.39e+00
epoch: 27   trainLoss: 2.8079e-04   valLoss:2.6844e-04  time: 1.38e+00
epoch: 28   trainLoss: 2.3821e-04   valLoss:2.7355e-04  time: 1.38e+00
epoch: 29   trainLoss: 2.1119e-04   valLoss:2.4250e-04  time: 1.39e+00
epoch: 30   trainLoss: 1.9689e-04   valLoss:2.2081e-04  time: 1.37e+00
epoch: 31   trainLoss: 1.7530e-04   valLoss:1.9640e-04  time: 1.38e+00
epoch:

epoch: 33   trainLoss: 1.6143e-04   valLoss:2.0248e-04  time: 1.36e+00
epoch: 34   trainLoss: 1.8597e-04   valLoss:2.3329e-04  time: 1.39e+00
epoch: 35   trainLoss: 1.4909e-04   valLoss:2.3027e-04  time: 1.37e+00
epoch: 36   trainLoss: 1.9278e-04   valLoss:3.1618e-04  time: 1.38e+00
epoch: 37   trainLoss: 2.0603e-04   valLoss:2.4822e-04  time: 1.38e+00
epoch: 38   trainLoss: 1.9519e-04   valLoss:3.0668e-04  time: 1.38e+00
epoch: 39   trainLoss: 1.9434e-04   valLoss:2.1384e-04  time: 1.38e+00
epoch: 40   trainLoss: 1.6047e-04   valLoss:2.3379e-04  time: 1.37e+00
epoch: 41   trainLoss: 1.7518e-04   valLoss:1.8848e-04  time: 1.38e+00
epoch: 42   trainLoss: 1.7148e-04   valLoss:2.1829e-04  time: 1.38e+00
epoch: 43   trainLoss: 1.5766e-04   valLoss:1.9207e-04  time: 1.36e+00
epoch: 44   trainLoss: 1.7516e-04   valLoss:2.2328e-04  time: 1.38e+00
epoch: 45   trainLoss: 1.8056e-04   valLoss:2.3076e-04  time: 1.37e+00
epoch: 46   trainLoss: 1.7187e-04   valLoss:2.0350e-04  time: 1.38e+00
epoch:

epoch: 48   trainLoss: 1.7613e-04   valLoss:2.5506e-04  time: 1.38e+00
epoch: 49   trainLoss: 2.0897e-04   valLoss:1.6411e-04  time: 1.37e+00
epoch: 50   trainLoss: 1.5314e-04   valLoss:1.3917e-04  time: 1.38e+00
epoch: 51   trainLoss: 1.5180e-04   valLoss:1.5048e-04  time: 1.38e+00
epoch: 52   trainLoss: 1.2617e-04   valLoss:1.1885e-04  time: 1.37e+00
epoch: 53   trainLoss: 1.2125e-04   valLoss:1.1427e-04  time: 1.37e+00
epoch: 54   trainLoss: 1.1028e-04   valLoss:1.5378e-04  time: 1.37e+00
epoch: 55   trainLoss: 1.3070e-04   valLoss:1.0152e-04  time: 1.38e+00
epoch: 56   trainLoss: 1.2761e-04   valLoss:1.1672e-04  time: 1.37e+00
epoch: 57   trainLoss: 1.1555e-04   valLoss:1.0928e-04  time: 1.39e+00
epoch: 58   trainLoss: 1.1878e-04   valLoss:1.1375e-04  time: 1.38e+00
epoch: 59   trainLoss: 1.1859e-04   valLoss:1.0365e-04  time: 1.38e+00
epoch: 60   trainLoss: 1.1910e-04   valLoss:1.0609e-04  time: 1.38e+00
epoch: 61   trainLoss: 1.1333e-04   valLoss:1.1802e-04  time: 1.37e+00
epoch:

In [13]:
df.groupby(['Standard Scaler', 'Log Transform']).mean()[['Train MSE', 'Test MSE']]

Train MSE  Test MSE
Standard Scaler Log Transform                     
False           False           0.000101  0.000118
                True            0.000045  0.000058
True            False           0.000011  0.000020
                True            0.000015  0.000025

## Should binary constraints be used as features?

In [6]:
saveDir = './results/modelTuning/constraintStudy01/'

# build training plan
options = [True, False]
seedList = [1,2,3]
trainConfigs = []
for uc in options:
        for s in seedList:
            trainConfigs.append({'Include supports': uc, 'Trial': s})

# train models
for i,c in enumerate(trainConfigs):
    print(c)
    trainData, valData, testData = partitionGraphList(allGraphs, seed=c['Trial'])
    gcn = FeaStNet(architecture='L16/C32/C64/C128/C256/C512/C256/C128/L64/L2', heads=8, useConstraints=c['Include supports'])
    
    t = time()
    history = gcn.trainModel(trainData, valData, 
                             epochs=100,
                             lr=1e-3,
                             batchSize=256, 
                             flatten=True, 
                             logTrans=False, 
                             ssTrans=True, 
                             saveDir=saveDir+f'/{i:03}/')
    
    # test
    trainRes = gcn.testModel(trainData)
    c['Train MSE'] = trainRes['mse']
    
    testRes = gcn.testModel(testData)
    c['Test MSE'] = testRes['mse']

    # log
    df = pd.DataFrame(trainConfigs)
    df.to_csv(saveDir+'testResults.csv', index=False)

{'Include supports': True, 'Trial': 1}
epoch: 0   trainLoss: 9.2956e-01   valLoss:8.8774e-01  time: 1.51e+00
epoch: 1   trainLoss: 7.8206e-01   valLoss:8.9228e-01  time: 1.44e+00
epoch: 2   trainLoss: 6.8264e-01   valLoss:9.0731e-01  time: 1.47e+00
epoch: 3   trainLoss: 6.0188e-01   valLoss:9.4842e-01  time: 1.44e+00
epoch: 4   trainLoss: 5.4546e-01   valLoss:1.0156e+00  time: 1.45e+00
epoch: 5   trainLoss: 4.6810e-01   valLoss:1.2122e+00  time: 1.45e+00
epoch: 6   trainLoss: 4.0749e-01   valLoss:1.2891e+00  time: 1.46e+00
epoch: 7   trainLoss: 3.8810e-01   valLoss:8.4728e-01  time: 1.44e+00
epoch: 8   trainLoss: 3.7031e-01   valLoss:5.3095e-01  time: 1.46e+00
epoch: 9   trainLoss: 2.9396e-01   valLoss:4.0760e-01  time: 1.45e+00
epoch: 10   trainLoss: 2.9250e-01   valLoss:4.6141e-01  time: 1.46e+00
epoch: 11   trainLoss: 2.5731e-01   valLoss:3.9848e-01  time: 1.45e+00
epoch: 12   trainLoss: 2.2300e-01   valLoss:5.4053e-01  time: 1.44e+00
epoch: 13   trainLoss: 1.9837e-01   valLoss:4.70

epoch: 15   trainLoss: 1.4983e-01   valLoss:3.5083e-01  time: 1.45e+00
epoch: 16   trainLoss: 1.3091e-01   valLoss:4.3770e-01  time: 1.45e+00
epoch: 17   trainLoss: 1.3478e-01   valLoss:2.7031e-01  time: 1.45e+00
epoch: 18   trainLoss: 1.4006e-01   valLoss:2.9501e-01  time: 1.45e+00
epoch: 19   trainLoss: 1.3884e-01   valLoss:3.5274e-01  time: 1.44e+00
epoch: 20   trainLoss: 1.1794e-01   valLoss:5.9289e-01  time: 1.42e+00
epoch: 21   trainLoss: 1.0990e-01   valLoss:4.7204e-01  time: 1.43e+00
epoch: 22   trainLoss: 1.0254e-01   valLoss:2.3180e-01  time: 1.41e+00
epoch: 23   trainLoss: 9.7090e-02   valLoss:2.6919e-01  time: 1.42e+00
epoch: 24   trainLoss: 9.8171e-02   valLoss:2.1661e-01  time: 1.44e+00
epoch: 25   trainLoss: 8.2323e-02   valLoss:2.9395e-01  time: 1.44e+00
epoch: 26   trainLoss: 1.0309e-01   valLoss:2.4054e-01  time: 1.45e+00
epoch: 27   trainLoss: 7.1204e-02   valLoss:1.8972e-01  time: 1.43e+00
epoch: 28   trainLoss: 7.7214e-02   valLoss:1.5812e-01  time: 1.42e+00
epoch:

epoch: 30   trainLoss: 6.3762e-02   valLoss:2.4208e-01  time: 1.44e+00
epoch: 31   trainLoss: 6.8679e-02   valLoss:1.4111e-01  time: 1.45e+00
epoch: 32   trainLoss: 7.0623e-02   valLoss:1.7667e-01  time: 1.45e+00
epoch: 33   trainLoss: 1.0996e-01   valLoss:1.4289e-01  time: 1.45e+00
epoch: 34   trainLoss: 7.5194e-02   valLoss:2.1258e-01  time: 1.44e+00
epoch: 35   trainLoss: 7.2813e-02   valLoss:1.2371e-01  time: 1.43e+00
epoch: 36   trainLoss: 5.4709e-02   valLoss:2.3921e-01  time: 1.46e+00
epoch: 37   trainLoss: 6.9934e-02   valLoss:1.2490e-01  time: 1.44e+00
epoch: 38   trainLoss: 7.8079e-02   valLoss:1.3457e-01  time: 1.44e+00
epoch: 39   trainLoss: 5.2162e-02   valLoss:2.0210e-01  time: 1.44e+00
epoch: 40   trainLoss: 5.8321e-02   valLoss:1.1247e-01  time: 1.45e+00
epoch: 41   trainLoss: 4.6632e-02   valLoss:1.3367e-01  time: 1.44e+00
epoch: 42   trainLoss: 4.6031e-02   valLoss:1.3047e-01  time: 1.44e+00
epoch: 43   trainLoss: 4.0839e-02   valLoss:1.1521e-01  time: 1.44e+00
epoch:

epoch: 45   trainLoss: 5.9468e-02   valLoss:1.6125e+00  time: 1.41e+00
epoch: 46   trainLoss: 5.4851e-02   valLoss:2.0280e+00  time: 1.43e+00
epoch: 47   trainLoss: 4.8517e-02   valLoss:1.6561e+00  time: 1.42e+00
epoch: 48   trainLoss: 4.8831e-02   valLoss:2.1328e+00  time: 1.44e+00
epoch: 49   trainLoss: 4.1388e-02   valLoss:1.1929e+00  time: 1.41e+00
epoch: 50   trainLoss: 8.5133e-02   valLoss:1.5388e+00  time: 1.40e+00
epoch: 51   trainLoss: 1.2463e-01   valLoss:1.1698e+00  time: 1.43e+00
epoch: 52   trainLoss: 7.1655e-02   valLoss:1.7201e+00  time: 1.42e+00
epoch: 53   trainLoss: 7.9318e-02   valLoss:1.1419e+00  time: 1.44e+00
epoch: 54   trainLoss: 7.4589e-02   valLoss:2.2353e+00  time: 1.42e+00
epoch: 55   trainLoss: 6.8950e-02   valLoss:1.6084e+00  time: 1.40e+00
epoch: 56   trainLoss: 6.8890e-02   valLoss:9.7055e-01  time: 1.40e+00
epoch: 57   trainLoss: 6.9102e-02   valLoss:8.1215e-01  time: 1.42e+00
epoch: 58   trainLoss: 7.3943e-02   valLoss:8.7996e-01  time: 1.40e+00
epoch:

epoch: 60   trainLoss: 5.0002e-02   valLoss:9.3538e-02  time: 1.41e+00
epoch: 61   trainLoss: 6.4109e-02   valLoss:1.4600e-01  time: 1.42e+00
epoch: 62   trainLoss: 6.8166e-02   valLoss:1.3919e-01  time: 1.42e+00
epoch: 63   trainLoss: 6.8260e-02   valLoss:1.9555e-01  time: 1.41e+00
epoch: 64   trainLoss: 6.4576e-02   valLoss:8.6512e-02  time: 1.41e+00
epoch: 65   trainLoss: 6.9540e-02   valLoss:4.1176e-01  time: 1.41e+00
epoch: 66   trainLoss: 7.8213e-02   valLoss:2.7281e-01  time: 1.43e+00
epoch: 67   trainLoss: 7.4960e-02   valLoss:2.5329e-01  time: 1.42e+00
epoch: 68   trainLoss: 7.4178e-02   valLoss:1.8541e-01  time: 1.41e+00
epoch: 69   trainLoss: 7.4322e-02   valLoss:2.0110e-01  time: 1.42e+00
epoch: 70   trainLoss: 9.5201e-02   valLoss:1.2691e-01  time: 1.39e+00
epoch: 71   trainLoss: 6.1507e-02   valLoss:1.4979e-01  time: 1.40e+00
epoch: 72   trainLoss: 6.6276e-02   valLoss:2.6729e-01  time: 1.41e+00
epoch: 73   trainLoss: 5.9692e-02   valLoss:1.2589e-01  time: 1.39e+00
epoch:

epoch: 75   trainLoss: 6.8712e-02   valLoss:1.5140e-01  time: 1.41e+00
epoch: 76   trainLoss: 5.6474e-02   valLoss:1.5518e-01  time: 1.42e+00
epoch: 77   trainLoss: 5.5010e-02   valLoss:1.0919e-01  time: 1.41e+00
epoch: 78   trainLoss: 4.1666e-02   valLoss:1.1899e-01  time: 1.43e+00
epoch: 79   trainLoss: 4.6990e-02   valLoss:1.6303e-01  time: 1.39e+00
epoch: 80   trainLoss: 5.2017e-02   valLoss:1.2589e-01  time: 1.43e+00
epoch: 81   trainLoss: 5.6781e-02   valLoss:1.6530e-01  time: 1.41e+00
epoch: 82   trainLoss: 5.4946e-02   valLoss:1.0056e-01  time: 1.40e+00
epoch: 83   trainLoss: 5.1042e-02   valLoss:1.2232e-01  time: 1.39e+00
epoch: 84   trainLoss: 7.2199e-02   valLoss:2.0883e-01  time: 1.43e+00
epoch: 85   trainLoss: 6.0779e-02   valLoss:9.9660e-02  time: 1.43e+00
epoch: 86   trainLoss: 5.5245e-02   valLoss:1.0420e-01  time: 1.40e+00
epoch: 87   trainLoss: 5.3951e-02   valLoss:1.1695e-01  time: 1.39e+00
epoch: 88   trainLoss: 4.0750e-02   valLoss:1.2711e-01  time: 1.41e+00
epoch:

In [7]:
df.groupby(['Include supports']).mean()[['Train MSE', 'Test MSE']]

,Train MSE,Test MSE
Include supports,,
False,0.000014,0.000027
True,0.000009,0.000017


In [8]:
df

,Include supports,Trial,Train MSE,Test MSE
0,True,1,0.000007,0.000016
1,True,2,0.000010,0.000016
2,True,3,0.000010,0.000019
3,False,1,0.000026,0.000045
4,False,2,0.000007,0.000017
5,False,3,0.000009,0.000019
